In [1]:
from src.dataset import ATPMatchesDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn as nn

import torch

In [2]:
from sklearn.preprocessing import MinMaxScaler
import torch

class StandardizeTransform:
    def __init__(self, numerical_indices, fit_data=None):
        """
        Args:
            numerical_indices: List of indices for numerical features to standardize
            fit_data: Data to fit the scaler on (typically training data features)
        """
        self.numerical_indices = numerical_indices
        self.scaler = MinMaxScaler()
        if fit_data is not None:
            # Extract only numerical features for fitting
            numerical_data = fit_data[:, numerical_indices]
            self.scaler.fit(numerical_data)

    def __call__(self, sample):
        features = sample['features'].clone()

        # Extract numerical features, transform, and put back
        numerical_features = features[self.numerical_indices]
        standardized = torch.tensor(
            self.scaler.transform(numerical_features.unsqueeze(0).numpy())[0],
            dtype=torch.float32
        )
        features[self.numerical_indices] = standardized

        return {'features': features, 'target': sample['target']}

In [3]:
dataset = ATPMatchesDataset('data/processed/atp_matches_2000_2024_final.csv')

train_loader = DataLoader(dataset.TRAIN, batch_size=32, shuffle=True, num_workers=4)
dev_loader = DataLoader(dataset.DEV, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset.TEST, batch_size=32, shuffle=False, num_workers=4)

In [4]:

class ATPMatchPredictor(nn.Module):
    def __init__(self, input_size, hidden_sizes=[128, 64, 32]):
        super(ATPMatchPredictor, self).__init__()

        layers = []
        prev_size = input_size

        # Hidden layers
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.3))
            prev_size = hidden_size

        # Output layer (binary classification)
        layers.append(nn.Linear(prev_size, 1))
        layers.append(nn.Sigmoid())

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [5]:
# Check if CUDA is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Get input size from first batch
sample_batch = next(iter(train_loader))
features = sample_batch['features']
labels = sample_batch['target']
input_size = features.shape[1]

print(f"Input size: {input_size}")

# Initialize model and move to device
model = ATPMatchPredictor(input_size=input_size, hidden_sizes=[128, 64, 32])
model = model.to(device)
print(model)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for batch in train_loader:
        features = batch['features'].to(device)  # Move to device
        labels = batch['target'].float().to(device)  # Move to device

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs.squeeze(), labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    dev_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dev_loader:
            features = batch['features'].to(device)  # Move to device
            labels = batch['target'].float().to(device)

            outputs = model(features)
            loss = criterion(outputs.squeeze(), labels)
            dev_loss += loss.item()

            predicted = (outputs.squeeze() > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss/len(train_loader):.4f}, '
          f'Dev Loss: {dev_loss/len(dev_loader):.4f}, '
          f'Dev Accuracy: {accuracy:.2f}%')


Input size: 216
ATPMatchPredictor(
  (network): Sequential(
    (0): Linear(in_features=216, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=32, out_features=1, bias=True)
    (10): Sigmoid()
  )
)
Epoch [1/50], Train Loss: 0.5824, Dev Loss: 0.5462, Dev Accuracy: 70.91%
